https://drive.google.com/drive/u/0/folders/1X_ag69ToGRCF32K2a099pgvvNMg0j8nq

In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
#BM Talos Single headed network year 1 hyperparameter search w/results
#full hyperparameter seaarch talos DNN/single headed(SH) model

#months 1-7 search

import numpy as np
import pandas as pd
from pandas import concat
import matplotlib.pyplot as plt
from functools import reduce
import importlib
import datetime as dt
from datetime import datetime
from math import floor
import seaborn as sns
from math import sqrt


#tensorflow &keras
import tensorflow as tf
from keras.layers import Input
from keras.models import Model
from tensorflow.keras.layers import GRU
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras import optimizers
from tensorflow.keras import initializers
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
from keras.layers import concatenate
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.wrappers.scikit_learn import KerasClassifier
from keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.callbacks import EarlyStopping
from keras.layers import LeakyReLU
LeakyReLU = LeakyReLU(alpha=0.1)
from keras.activations import *

from keras.layers import LeakyReLU
LeakyReLU = LeakyReLU(alpha=0.1)

#sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
from sklearn import metrics
from sklearn import preprocessing as prep
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import accuracy_score, make_scorer
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import make_scorer, accuracy_score, mean_absolute_error
from sklearn.metrics import mean_squared_error as MSE
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
import talos 
import tensorflow.keras.backend as K
import pandas as pd
from pathlib import Path
import altair as alt
import numpy as np
import pandas as pd
from pyomo.environ import *

date_format = "%d/%m/%Y %H:%M"
date_parse = lambda date: dt.datetime.strptime(date, date_format)
dat = pd.read_csv("/home/ciaran/Documents/DAM_T_D.csv", index_col="DeliveryPeriod", parse_dates=True, date_parser=date_parse)

# .iloc[13104:21864,:]
Y=dat.iloc[:, 0:1]
Y

2023-05-01 01:38:20.655461: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-01 01:38:20.730551: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-01 01:38:21.125021: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-05-01 01:38:21.125059: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

,EURPrices
DeliveryPeriod,
2018-12-03 00:00:00,55.000
2018-12-03 01:00:00,50.830
2018-12-03 02:00:00,50.260
2018-12-03 03:00:00,47.000
2018-12-03 04:00:00,46.937
...,...
2022-05-13 18:00:00,303.000
2022-05-13 19:00:00,291.400
2022-05-13 20:00:00,274.270


In [2]:
def read_filter_lbmp(path):
    date_format = "%d/%m/%Y %H:%M"
    date_parse = lambda date: dt.datetime.strptime(date, date_format)
    df = pd.read_csv(path)
    return df

def read_all_nyc(data_path):
    """
    Reads and combines individual LBMP data files.
    
    Parameters
    ----------
    data_path : Path object
        This is a pathlib Path object pointing to the LBMP data folder with
        .csv files.
    
    Returns
    -------
    DataFrame
        df with 4 columns (time stamp, name of node, LBMP, hour of year)
    """

    fnames = data_path.glob('DAM_T_D.csv')
    dfs = [read_filter_lbmp(name) for name in fnames]
    df = pd.concat(dfs)
    df=df.iloc[13104:21864,:]
    df.sort_values('DeliveryPeriod', inplace=False)
    df.reset_index(inplace=True, drop=True)
    df['hour'] = df.index

    return df

data_path = Path.cwd() / 'Documents'
df=read_all_nyc(data_path=data_path)




def model_to_df(model, first_hour, last_hour):
    """
    Create a dataframe with hourly charge, discharge, state of charge, and
    price columns from a pyomo model. Only uses data from between the first
    (inclusive) and last (exclusive) hours.
    
    Parameters
    ----------
    model : pyomo model
        Model that has been solved
    first_hour : int
        First hour of data to keep
    last_hour: int
        The final hour of data to keep
    
    Returns
    -------
    dataframe
        
    """
    # Need to increase the first & last hour by 1 because of pyomo indexing 
    # and add 1 to the value of last model hour because of the range
    # second model hour by 1
    hours = range(model.T[first_hour + 1], model.T[last_hour + 1] + 1)
    Ein = [value(model.Ein[i]) for i in hours]
    Eout = [value(model.Eout[i]) for i in hours]
    EURPrices = [model.P.extract_values()[None][i] for i in hours]
    charge_state = [value(model.S[i]) for i in hours]

    df_dict = dict(
        hour=hours,
        Ein=Ein,
        Eout=Eout,
        EURPrices=EURPrices,
        charge_state=charge_state
    )

    df = pd.DataFrame(df_dict)

    return df

def optimize_year(df, first_model_hour=0, last_model_hour=30180):
    """
    Optimize the charge/discharge behavior of a battery storage unit over a
    full year. Assume perfect foresight of electricity prices. The battery
    has a discharge constraint equal to its storage capacity and round-trip
    efficiency of 85%.
    
    Parameters
    ----------
    df : dataframe
        dataframe with columns of hourly LBMP and the hour of the year
    first_model_hour : int, optional
        Set the first hour of the year to be considered in the optimization
        (the default is 0)
    last_model_hour : int, optional
        Set the last hour of the year to be considered in the optimization (the
        default is 8759)
    
    Returns
    -------
    dataframe
        hourly state of charge, charge/discharge behavior, lbmp, and time stamp
    """

    #Filter the data
    df = df.loc[first_model_hour:last_model_hour, :]

    model = ConcreteModel()

    # Define model parameters
    model.T = Set(doc='hour of year', initialize=df.hour.tolist(), ordered=True)
    model.Rmax = Param(initialize=100,
                       doc='Max rate of power flow (kW) in or out')
    model.Smax = Param(initialize=200, doc='Max storage (kWh)')
    model.Dmax = Param(initialize=200, doc='Max discharge in 24 hour period')
    model.P = Param(initialize=df.EURPrices.tolist(), doc='EURPrices for each hour')
    eta = 0.85 # Round trip storage efficiency

    # Charge, discharge, and state of charge
    # Could use bounds for the first 2 instead of constraints
    model.Ein =  Var(model.T,  domain=NonNegativeReals)
    model.Eout = Var(model.T,  domain=NonNegativeReals)
    model.S =    Var(model.T,  bounds=(0, model.Smax))


    #Set all constraints
    def storage_state(model, t):
        'Storage changes with flows in/out and efficiency losses'
        # Set first hour state of charge to half of max
        if t == model.T.first():
            return model.S[t] == model.Smax / 2
        else:
            return (model.S[t] == (model.S[t-1] 
                                + (model.Ein[t-1] * np.sqrt(eta)) 
                                - (model.Eout[t-1] / np.sqrt(eta))))

    model.charge_state = Constraint(model.T, rule=storage_state)

    def discharge_constraint(model, t):
        "Maximum dischage within a single hour"
        return model.Eout[t] <= model.Rmax

    model.discharge = Constraint(model.T, rule=discharge_constraint)

    def charge_constraint(model, t):
        "Maximum charge within a single hour"
        return model.Ein[t] <= model.Rmax

    model.charge = Constraint(model.T, rule=charge_constraint)

    # Without a constraint the model would discharge in the final hour
    # even when SOC was 0.
    def positive_charge(model, t):
        'Limit discharge to the amount of charge in battery, including losses'
        return model.Eout[t] <= model.S[t] / np.sqrt(eta)
    model.positive_charge = Constraint(model.T, rule=positive_charge)

    def discharge_limit(model, t):
        "Limit on discharge within a 24 hour period"
        max_t = model.T.last()

        # Check all t until the last 24 hours
        # No need to check with < 24 hours remaining because the constraint is
        # already in place for a larger number of hours
        if t < max_t - 24:
            return sum(model.Eout[i] for i in range(t, t+24)) <= model.Dmax
        else:
            return Constraint.Skip

    model.limit_out = Constraint(model.T, rule=discharge_limit)

    # Define the battery income, expenses, and profit
    income = sum(df.loc[t, 'EURPrices'] * model.Eout[t] for t in model.T)
    expenses = sum(df.loc[t, 'EURPrices'] * model.Ein[t] for t in model.T)
    profit = income - expenses
    model.objective = Objective(expr=profit, sense=maximize)

    # Solve the model
    solver = SolverFactory('glpk')
    solver.solve(model)

    results_df = model_to_df(model, first_hour=first_model_hour,
                             last_hour=last_model_hour)
    results_df['DeliveryPeriod'] = df.loc[:, 'DeliveryPeriod']

    return results_df

df

,DeliveryPeriod,EURPrices,hour
0,01/06/2020 00:00,24.65,0
1,01/06/2020 01:00,21.59,1
2,01/06/2020 02:00,18.29,2
3,01/06/2020 03:00,14.12,3
4,01/06/2020 04:00,11.58,4
...,...,...,...
8755,31/05/2021 19:00,105.00,8755
8756,31/05/2021 20:00,94.50,8756
8757,31/05/2021 21:00,84.90,8757
8758,31/05/2021 22:00,87.55,8758


In [3]:
results_df = optimize_year(df, first_model_hour=0, last_model_hour=8759)

results_df['revenue'] = results_df.Eout * results_df.EURPrices / 1000
results_df['charge_cost'] = results_df.Ein * results_df.EURPrices  / 1000
results_df['profit'] = results_df.revenue - results_df.charge_cost

total_revenue = results_df.revenue.sum()
total_charge_cost = results_df.charge_cost.sum()
total_discharge = results_df.Eout.sum()

print('Annual revenue was ${:,.2f}'.format(total_revenue))
print('Annual charging cost was ${:,.2f}'.format(total_charge_cost))
print('Annual discharged throughput was {:,.2f} kWh'.format(total_discharge))

    default domain for Param objects is 'Any'.  However, we will be changing
    that default to 'Reals' in the future.  If you really intend the domain of
    this Paramto be 'Any', you can suppress this warning by explicitly
    specifying 'within=Any' to the Param constructor. (deprecated in 5.6.9,
    will be removed in (or after) 6.0) (called from
    /home/ciaran/anaconda3/lib/python3.9/site-
    packages/pyomo/core/base/indexed_component.py:717)
    (ordered) position is deprecated.  Please use at()  (deprecated in 6.1,
    will be removed in (or after) 7.0) (called from
    /tmp/ipykernel_6083/926927306.py:62)
Annual revenue was $6,439.00
Annual charging cost was $2,422.44
Annual discharged throughput was 67,262.26 kWh


In [4]:
results_df['DeliveryPeriod'] = pd.to_datetime(results_df['DeliveryPeriod'])
results_df['week'] = results_df['DeliveryPeriod'].dt.week
results_df['month'] = results_df['DeliveryPeriod'].dt.month
results_df['hour_of_day'] = results_df['DeliveryPeriod'].dt.hour
results_df.loc[results_df['DeliveryPeriod'] >= '2020-01-06', :].groupby('week')['profit'].sum().idxmax()
results_df

,hour,Ein,Eout,EURPrices,charge_state,DeliveryPeriod,revenue,charge_cost,profit,week,month,hour_of_day
0,0,0.0,15.609111,24.65,100.000000,2020-01-06 00:00:00,0.384765,0.000,0.384765,2,1,0
1,1,0.0,0.000000,21.59,83.069542,2020-01-06 01:00:00,0.000000,0.000,0.000000,2,1,1
2,2,0.0,0.000000,18.29,83.069542,2020-01-06 02:00:00,0.000000,0.000,0.000000,2,1,2
3,3,0.0,0.000000,14.12,83.069542,2020-01-06 03:00:00,0.000000,0.000,0.000000,2,1,3
4,4,100.0,0.000000,11.58,83.069542,2020-01-06 04:00:00,0.000000,1.158,-1.158000,2,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...
8755,8755,0.0,0.000000,105.00,0.000000,2021-05-31 19:00:00,0.000000,0.000,0.000000,22,5,19
8756,8756,0.0,0.000000,94.50,0.000000,2021-05-31 20:00:00,0.000000,0.000,0.000000,22,5,20
8757,8757,0.0,0.000000,84.90,0.000000,2021-05-31 21:00:00,0.000000,0.000,0.000000,22,5,21
8758,8758,0.0,0.000000,87.55,0.000000,2021-05-31 22:00:00,0.000000,0.000,0.000000,22,5,22


In [5]:
data = results_df.loc[(results_df['DeliveryPeriod'] >= '2020-01-06') &
                      (results_df.week == 22), :].copy()
data.loc[:, 'dispatch'] = data.Ein - data.Eout
dispatch_data = pd.melt(data, id_vars='DeliveryPeriod', value_vars=['Eout', 'Ein'], var_name='Dispatch')

color_scale = alt.Scale(
            domain=['Ein', 'Eout'],
            range=['#f99820', '#2081f9']
        )

dispatch = alt.Chart(dispatch_data).mark_line().encode(
    x='DeliveryPeriod:T',
    y=alt.Y('value:Q', axis=alt.Axis(title='Electricity in/out (kWh)')),
    color=alt.Color('Dispatch:N', scale=color_scale)
).properties(
    height=200
)

EURPrices = alt.Chart(data).mark_circle().encode(
    x='DeliveryPeriod:T',
    y=alt.Y('EURPrices:Q', axis=alt.Axis(title='ISEM EURPrices ($)')),
    color=alt.Color('dispatch:Q', scale=alt.Scale(scheme='blueorange')),
    tooltip='dispatch:Q'
).properties(
    height=200
)

alt.vconcat(
    dispatch,
    EURPrices
)

alt.VConcatChart(...)

In [6]:
alt.Chart(results_df.sample(5000)).mark_line().encode(
    x='month:O',
    y='sum(profit):Q'
).properties(
    height=300,
    width=500
)


alt.Chart(...)

In [7]:
date_format = "%m/%d/%Y %H:%M"
date_parse = lambda date: dt.datetime.strptime(date, date_format)
dat = pd.read_csv("/home/ciaran/Documents/BM_data.csv", index_col="SettlementPeriod", parse_dates=True,  date_parser=date_parse)

dat = dat.drop(["index"], axis=1)
dat = pd.DataFrame(dat)
dat = dat.bfill(axis='rows')
dat = dat.ffill(axis='rows')
dat = dat._get_numeric_data()

Y=dat.iloc[:, 0:1]

In [8]:
def read_filter_lbmp(path):
    date_format = "%m/%d/%Y %H:%M"
    date_parse = lambda date: dt.datetime.strptime(date, date_format)
    df = pd.read_csv(path)
    return df

def read_all_nyc(data_path):
    """
    Reads and combines individual LBMP data files.
    
    Parameters
    ----------
    data_path : Path object
        This is a pathlib Path object pointing to the LBMP data folder with
        .csv files.
    
    Returns
    -------
    DataFrame
        df with 4 columns (time stamp, name of node, LBMP, hour of year)
    """

    fnames = data_path.glob('BM_data.csv')
    dfs = [read_filter_lbmp(name) for name in fnames]
    df = pd.concat(dfs)
    df=df.iloc[8738:26258, :]
    df.sort_values('SettlementPeriod', inplace=False)
    df.reset_index(inplace=True, drop=True)
    df['hour'] = df.index

    return df


data_path = Path.cwd() / 'Documents'
df=read_all_nyc(data_path=data_path)

def model_to_df(model, first_hour, last_hour):
    """
    Create a dataframe with hourly charge, discharge, state of charge, and
    price columns from a pyomo model. Only uses data from between the first
    (inclusive) and last (exclusive) hours.
    
    Parameters
    ----------
    model : pyomo model
        Model that has been solved
    first_hour : int
        First hour of data to keep
    last_hour: int
        The final hour of data to keep
    
    Returns
    -------
    dataframe
        
    """
    # Need to increase the first & last hour by 1 because of pyomo indexing 
    # and add 1 to the value of last model hour because of the range
    # second model hour by 1
    hours = range(model.T[first_hour + 1], model.T[last_hour + 1] + 1)
    Ein = [value(model.Ein[i]) for i in hours]
    Eout = [value(model.Eout[i]) for i in hours]
    lag_2y = [model.P.extract_values()[None][i] for i in hours]
    charge_state = [value(model.S[i]) for i in hours]

    df_dict = dict(
        hour=hours,
        Ein=Ein,
        Eout=Eout,
        lag_2y=lag_2y,
        charge_state=charge_state
    )

    df = pd.DataFrame(df_dict)

    return df

def optimize_year(df, first_model_hour=0, last_model_hour=30180):
    """
    Optimize the charge/discharge behavior of a battery storage unit over a
    full year. Assume perfect foresight of electricity prices. The battery
    has a discharge constraint equal to its storage capacity and round-trip
    efficiency of 85%.
    
    Parameters
    ----------
    df : dataframe
        dataframe with columns of hourly LBMP and the hour of the year
    first_model_hour : int, optional
        Set the first hour of the year to be considered in the optimization
        (the default is 0)
    last_model_hour : int, optional
        Set the last hour of the year to be considered in the optimization (the
        default is 8759)
    
    Returns
    -------
    dataframe
        hourly state of charge, charge/discharge behavior, lbmp, and time stamp
    """

    #Filter the data
    df = df.loc[first_model_hour:last_model_hour, :]

    model = ConcreteModel()

    # Define model parameters
    model.T = Set(doc='hour of year', initialize=df.hour.tolist(), ordered=True)
    model.Rmax = Param(initialize=100,
                       doc='Max rate of power flow (kW) in or out')
    model.Smax = Param(initialize=200, doc='Max storage (kWh)')
    model.Dmax = Param(initialize=200, doc='Max discharge in 24 hour period')
    model.P = Param(initialize=df.lag_2y.tolist(), doc='EURPrices for each hour')
    eta = 0.85 # Round trip storage efficiency

    # Charge, discharge, and state of charge
    # Could use bounds for the first 2 instead of constraints
    model.Ein =  Var(model.T,  domain=NonNegativeReals)
    model.Eout = Var(model.T,  domain=NonNegativeReals)
    model.S =    Var(model.T,  bounds=(0, model.Smax))


    #Set all constraints
    def storage_state(model, t):
        'Storage changes with flows in/out and efficiency losses'
        # Set first hour state of charge to half of max
        if t == model.T.first():
            return model.S[t] == model.Smax / 2
        else:
            return (model.S[t] == (model.S[t-1] 
                                + (model.Ein[t-1] * np.sqrt(eta)) 
                                - (model.Eout[t-1] / np.sqrt(eta))))

    model.charge_state = Constraint(model.T, rule=storage_state)

    def discharge_constraint(model, t):
        "Maximum dischage within a single hour"
        return model.Eout[t] <= model.Rmax

    model.discharge = Constraint(model.T, rule=discharge_constraint)

    def charge_constraint(model, t):
        "Maximum charge within a single hour"
        return model.Ein[t] <= model.Rmax

    model.charge = Constraint(model.T, rule=charge_constraint)

    # Without a constraint the model would discharge in the final hour
    # even when SOC was 0.
    def positive_charge(model, t):
        'Limit discharge to the amount of charge in battery, including losses'
        return model.Eout[t] <= model.S[t] / np.sqrt(eta)
    model.positive_charge = Constraint(model.T, rule=positive_charge)

    def discharge_limit(model, t):
        "Limit on discharge within a 24 hour period"
        max_t = model.T.last()

        # Check all t until the last 24 hours
        # No need to check with < 24 hours remaining because the constraint is
        # already in place for a larger number of hours
        if t < max_t - 24:
            return sum(model.Eout[i] for i in range(t, t+24)) <= model.Dmax
        else:
            return Constraint.Skip

    model.limit_out = Constraint(model.T, rule=discharge_limit)

    # Define the battery income, expenses, and profit
    income = sum(df.loc[t, 'lag_2y'] * model.Eout[t] for t in model.T)
    expenses = sum(df.loc[t, 'lag_2y'] * model.Ein[t] for t in model.T)
    profit = income - expenses
    model.objective = Objective(expr=profit, sense=maximize)

    # Solve the model
    solver = SolverFactory('glpk')
    solver.solve(model)

    results_df = model_to_df(model, first_hour=first_model_hour,
                             last_hour=last_model_hour)
    results_df['SettlementPeriod'] = df.loc[:, 'SettlementPeriod']

    return results_df

df



,index,SettlementPeriod,lag_2y,lag_3y,lag_4y,lag_5y,lag_6y,lag_7y,lag_8y,lag_9y,...,lag_-42x12,lag_-43x12,lag_-44x12,lag_-45x12,lag_-46x12,lag_-47x12,lag_-48x12,lag_-49x12,lag_-50x12,hour
0,8738,6/1/2020 0:00,15.60,15.87,24.93,31.50,13.91,17.66,17.87,16.30,...,107.0,107.0,106.0,106.0,185.0,91.0,-219.0,-514.0,-661.0,0
1,8739,6/1/2020 0:30,15.87,24.93,31.50,13.91,17.66,17.87,16.30,17.88,...,106.0,107.0,107.0,106.0,106.0,185.0,91.0,-219.0,-514.0,1
2,8740,6/1/2020 1:00,24.93,31.50,13.91,17.66,17.87,16.30,17.88,30.90,...,106.0,106.0,107.0,107.0,106.0,106.0,185.0,91.0,-219.0,2
3,8741,6/1/2020 1:30,31.50,13.91,17.66,17.87,16.30,17.88,30.90,19.95,...,107.0,106.0,106.0,107.0,107.0,106.0,106.0,185.0,91.0,3
4,8742,6/1/2020 2:00,13.91,17.66,17.87,16.30,17.88,30.90,19.95,137.66,...,205.0,107.0,106.0,106.0,107.0,107.0,106.0,106.0,185.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17515,26253,5/31/2021 21:30,58.45,60.00,59.93,58.67,56.81,42.38,38.58,43.03,...,155.0,100.0,120.0,320.0,220.0,281.0,242.0,222.0,245.0,17515
17516,26254,5/31/2021 22:00,60.00,59.93,58.67,56.81,42.38,38.58,43.03,36.82,...,168.0,155.0,100.0,120.0,320.0,220.0,281.0,242.0,222.0,17516
17517,26255,5/31/2021 22:30,59.93,58.67,56.81,42.38,38.58,43.03,36.82,39.54,...,148.0,168.0,155.0,100.0,120.0,320.0,220.0,281.0,242.0,17517
17518,26256,5/31/2021 23:00,58.67,56.81,42.38,38.58,43.03,36.82,39.54,40.81,...,92.0,148.0,168.0,155.0,100.0,120.0,320.0,220.0,281.0,17518


In [9]:
results_df = optimize_year(df, first_model_hour = 0, last_model_hour = 17519)

results_df['revenue'] = results_df.Eout * results_df.lag_2y / 1000
results_df['charge_cost'] = results_df.Ein * results_df.lag_2y  / 1000
results_df['profit'] = results_df.revenue - results_df.charge_cost

total_revenue = results_df.revenue.sum()
total_charge_cost = results_df.charge_cost.sum()
total_discharge = results_df.Eout.sum()

print('Annual revenue was ${:,.2f}'.format(total_revenue))
print('Annual charging cost was ${:,.2f}'.format(total_charge_cost))
print('Annual discharged throughput was {:,.2f} kWh'.format(total_discharge))

    (ordered) position is deprecated.  Please use at()  (deprecated in 6.1,
    will be removed in (or after) 7.0) (called from
    /tmp/ipykernel_6083/475632705.py:60)
Annual revenue was $15,523.44
Annual charging cost was $-379.85
Annual discharged throughput was 118,962.18 kWh


In [10]:
results_df['SettlementPeriod'] = pd.to_datetime(results_df['SettlementPeriod'])
results_df['week'] = results_df['SettlementPeriod'].dt.week
results_df['month'] = results_df['SettlementPeriod'].dt.month
results_df['hour_of_day'] = results_df['SettlementPeriod'].dt.hour
results_df.loc[results_df['SettlementPeriod'] >= '2020-06-01', :].groupby('week')['profit'].sum().idxmax()
results_df

,hour,Ein,Eout,lag_2y,charge_state,SettlementPeriod,revenue,charge_cost,profit,week,month,hour_of_day
0,0,8.465229,0.0,15.60,100.000000,2020-06-01 00:00:00,0.0,0.132058,-0.132058,23,6,0
1,1,0.000000,0.0,15.87,107.804555,2020-06-01 00:30:00,0.0,0.000000,0.000000,23,6,0
2,2,0.000000,0.0,24.93,107.804555,2020-06-01 01:00:00,0.0,0.000000,0.000000,23,6,1
3,3,0.000000,0.0,31.50,107.804555,2020-06-01 01:30:00,0.0,0.000000,0.000000,23,6,1
4,4,100.000000,0.0,13.91,107.804555,2020-06-01 02:00:00,0.0,1.391000,-1.391000,23,6,2
...,...,...,...,...,...,...,...,...,...,...,...,...
17515,17515,-0.000000,0.0,58.45,0.000000,2021-05-31 21:30:00,0.0,-0.000000,0.000000,22,5,21
17516,17516,0.000000,0.0,60.00,0.000000,2021-05-31 22:00:00,0.0,0.000000,0.000000,22,5,22
17517,17517,0.000000,0.0,59.93,0.000000,2021-05-31 22:30:00,0.0,0.000000,0.000000,22,5,22
17518,17518,0.000000,0.0,58.67,0.000000,2021-05-31 23:00:00,0.0,0.000000,0.000000,22,5,23


In [11]:
data = results_df.loc[(results_df['SettlementPeriod'] >= '2020-06-01') &
                      (results_df.week == 22), :].copy()
data.loc[:, 'dispatch'] = data.Ein - data.Eout
dispatch_data = pd.melt(data, id_vars='SettlementPeriod', value_vars=['Eout', 'Ein'], var_name='Dispatch')

color_scale = alt.Scale(
            domain=['Ein', 'Eout'],
            range=['#f99820', '#2081f9']
        )

dispatch = alt.Chart(dispatch_data).mark_line().encode(
    x='SettlementPeriod:T',
    y=alt.Y('value:Q', axis=alt.Axis(title='Electricity in/out (kWh)')),
    color=alt.Color('Dispatch:N', scale=color_scale)
).properties(
    height=200
)

EURPrices = alt.Chart(data).mark_circle().encode(
    x='SettlementPeriod:T',
    y=alt.Y('EURPrices:Q', axis=alt.Axis(title='ISEM EURPrices ($)')),
    color=alt.Color('dispatch:Q', scale=alt.Scale(scheme='blueorange')),
    tooltip='dispatch:Q'
).properties(
    height=200
)

alt.vconcat(
    dispatch,
    EURPrices
)

alt.VConcatChart(...)

In [12]:
alt.Chart(results_df.sample(5000)).mark_line().encode(
    x='month:O',
    y='sum(profit):Q'
).properties(
    height=300,
    width=500
)


alt.Chart(...)